In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('video_data.csv', error_bad_lines=False)

b'Skipping line 464716: expected 3 fields, saw 4\nSkipping line 464947: expected 3 fields, saw 4\nSkipping line 465187: expected 3 fields, saw 4\nSkipping line 465442: expected 3 fields, saw 4\nSkipping line 465648: expected 3 fields, saw 4\n'


In [3]:
df1.shape

(1333653, 3)

# •	VideoStart metric

1. Determined from the “events” column containing “206”
2. All rows without 206 should be discarded

In [4]:
df1.head(2)

,DateTime,VideoTitle,events
0,2017-01-11T00:00:31.000Z,App Web|Clips|a-current-affair;2016|William Ty...,"157,120,160,104,162,161,163,164,165,166,171,229"
1,2017-01-11T00:00:53.000Z,news| Shark attacks spearfisherman,"127,157,120,160,104,162,161,171,206"


In [4]:
Events_With_206 = df1.events.str.contains('206')

df_without_206 = df1[~Events_With_206].reset_index(drop=True)
df_with_206 = df1[Events_With_206].reset_index(drop=True)
print("The {} records has been deleted. \n\nThe new dataset has {} rows and {} columns.".format(df_without_206.shape[0],
                                                                                           df_with_206.shape[0],
                                                                                           df_with_206.shape[1]))
         

The 835544 records has been deleted. 

The new dataset has 498109 rows and 3 columns.


# •	DimDate

1. should go to the minute grain

In [30]:
df_with_206

,DateTime,VideoTitle,events
0,2017-01-11T00:00:53.000Z,news| Shark attacks spearfisherman,"127,157,120,160,104,162,161,171,206"
1,2017-01-11T00:03:51.000Z,news| Chilean navy films UFO,"127,157,120,160,104,162,161,171,206"
2,2017-01-11T00:02:42.000Z,news| Chilean navy films UFO,"157,120,160,104,162,161,171,206"
3,2017-01-11T00:04:07.000Z,news|Exclusive CCTV footage of Minto attack re...,"157,120,160,104,162,161,171,206"
4,2017-01-11T00:03:07.000Z,news|Blue shark washes up on St Kilda beach,"127,157,120,160,104,162,161,171,206"
...,...,...,...
498104,2017-01-11T23:46:00.000Z,App iPad|Episode|heartbeat;season-11|Ep 11 A G...,"501,508,507,509,121,105,120,160,162,171,206,15..."
498105,2017-01-11T23:59:46.000Z,App iPad|Live|Channel 9|WA,"501,508,507,509,121,105,120,160,162,171,206,15..."
498106,2017-01-11T23:57:57.000Z,App iPad|Episode|hawaii-life;season-7|Ep 4 Lif...,"501,508,507,509,121,105,120,160,162,171,206,15..."
498107,2017-01-11T23:56:13.000Z,App iPhone|Clip|a-current-affair;2017|Cruise i...,"501,508,507,509,121,105,120,160,162,171,206,15..."


In [54]:
df_with_206.DateTime = df_with_206.DateTime.apply(lambda x :x.replace('T', ' '))

df_with_206.DateTime = df_with_206.DateTime.apply(lambda x :x.replace('.000Z', ''))

df_with_206['DateTime'] = pd.to_datetime(df_with_206['DateTime'],format = "%Y/%m/%d %H:%M:%S")

In [55]:
df_with_206['DateTime'] = pd.to_datetime(df_with_206['DateTime'],format = "%Y/%m/%d %H:%M:%S")

# •	DimPlatform

1. Split VideoTitle by pipe ‘|’
2. If VideoTitle.split(‘|’)[0] contains something that looks like a platform (iPhone, Android Phone etc) then use that as the platform
3. If VideoTitle.split(‘|’)[0] doesn’t contain a platform but looks like a site, assume the platform is Desktop
4. If VideoTitle.split(‘|’).count = 1, discard the row. -->


In [46]:
# discard the row that len == 1

# df_without_206.VideoTitle = df_without_206.VideoTitle.apply(lambda x :x.replace('|', ' '))

df_with_206.VideoTitle = df_with_206.VideoTitle.str.split('|')

for index,row in df_without_206.iterrows():
    count = len(row['VideoTitle'])
    if count ==1:
        df_without_206=df_without_206.drop(index=index)
        

In [5]:
df_with_206.VideoTitle = df_with_206.VideoTitle.str.split('|')
df_with_206.head(2)

,DateTime,VideoTitle,events
0,2017-01-11T00:00:53.000Z,"[news, Shark attacks spearfisherman]","127,157,120,160,104,162,161,171,206"
1,2017-01-11T00:03:51.000Z,"[news, Chilean navy films UFO]","127,157,120,160,104,162,161,171,206"


In [56]:
df_with_206['Platform']=' '
df_with_206['Platform'][df_with_206['VideoTitle'].str.contains('iPhone')]='iPhone'
df_with_206['Platform'][df_with_206['VideoTitle'].str.contains('Android')]='Android'
df_with_206['Platform'][df_with_206['VideoTitle'].str.contains('iPad')]='Ipad'
df_with_206['Platform'][~df_with_206['VideoTitle'].str.contains('iPhone|Android|iPad')]='Desktop'


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing t

In [57]:
df_with_206['Platform'].value_counts()

Desktop    441764
Ipad        28883
Android     15621
iPhone      11841
Name: Platform, dtype: int64

# •	DimSite

1. Split VideoTitle by pipe |
2. If VideoTitle.split(‘|’).count = 1, discard the row.
3. If VideoTitle.split(‘|’)[0] looks like a site name, save the site name

# •	DimVideo

1. Last piece of VideoTitle.split(‘|’) contains the video title
2. You can ignore any middle pieces



In [59]:
df_with_206.VideoTitle = df_with_206.VideoTitle.str.split('|')
df_with_206

,DateTime,VideoTitle,events,Platform
0,2017-01-11 00:00:53,"[news, Shark attacks spearfisherman]","127,157,120,160,104,162,161,171,206",Desktop
1,2017-01-11 00:03:51,"[news, Chilean navy films UFO]","127,157,120,160,104,162,161,171,206",Desktop
2,2017-01-11 00:02:42,"[news, Chilean navy films UFO]","157,120,160,104,162,161,171,206",Desktop
3,2017-01-11 00:04:07,"[news, Exclusive CCTV footage of Minto attack ...","157,120,160,104,162,161,171,206",Desktop
4,2017-01-11 00:03:07,"[news, Blue shark washes up on St Kilda beach]","127,157,120,160,104,162,161,171,206",Desktop
...,...,...,...,...
498104,2017-01-11 23:46:00,"[App iPad, Episode, heartbeat;season-11, Ep 11...","501,508,507,509,121,105,120,160,162,171,206,15...",Ipad
498105,2017-01-11 23:59:46,"[App iPad, Live, Channel 9, WA]","501,508,507,509,121,105,120,160,162,171,206,15...",Ipad
498106,2017-01-11 23:57:57,"[App iPad, Episode, hawaii-life;season-7, Ep 4...","501,508,507,509,121,105,120,160,162,171,206,15...",Ipad
498107,2017-01-11 23:56:13,"[App iPhone, Clip, a-current-affair;2017, Crui...","501,508,507,509,121,105,120,160,162,171,206,15...",iPhone


In [60]:
title_list = []
site_name = []
for row in df_with_206['VideoTitle']:
    title_list.append(row[-1])
    site_name.append(row[0])
    
    
df_with_206['DimSite'] = pd.DataFrame(site_name)    
df_with_206['DimVideo'] = pd.DataFrame(title_list)
df_with_206.head()

,DateTime,VideoTitle,events,Platform,DimSite,DimVideo
0,2017-01-11 00:00:53,"[news, Shark attacks spearfisherman]","127,157,120,160,104,162,161,171,206",Desktop,news,Shark attacks spearfisherman
1,2017-01-11 00:03:51,"[news, Chilean navy films UFO]","127,157,120,160,104,162,161,171,206",Desktop,news,Chilean navy films UFO
2,2017-01-11 00:02:42,"[news, Chilean navy films UFO]","157,120,160,104,162,161,171,206",Desktop,news,Chilean navy films UFO
3,2017-01-11 00:04:07,"[news, Exclusive CCTV footage of Minto attack ...","157,120,160,104,162,161,171,206",Desktop,news,Exclusive CCTV footage of Minto attack released
4,2017-01-11 00:03:07,"[news, Blue shark washes up on St Kilda beach]","127,157,120,160,104,162,161,171,206",Desktop,news,Blue shark washes up on St Kilda beach


In [61]:
df_with_206.shape

(498109, 6)

In [1]:
df_with_206.to_csv('Cleaned_video_data.csv',index=0)

NameError: name 'df_with_206' is not defined

In [3]:
newlist = pd.read_csv("Cleaned_video_data.csv")
newlist.head()

,DateTime,VideoTitle,events,Platform,DimSite,DimVideo
0,2017-01-11 00:00:53,"['news', ' Shark attacks spearfisherman']","127,157,120,160,104,162,161,171,206",Desktop,news,Shark attacks spearfisherman
1,2017-01-11 00:03:51,"['news', ' Chilean navy films UFO']","127,157,120,160,104,162,161,171,206",Desktop,news,Chilean navy films UFO
2,2017-01-11 00:02:42,"['news', ' Chilean navy films UFO']","157,120,160,104,162,161,171,206",Desktop,news,Chilean navy films UFO
3,2017-01-11 00:04:07,"['news', 'Exclusive CCTV footage of Minto atta...","157,120,160,104,162,161,171,206",Desktop,news,Exclusive CCTV footage of Minto attack released
4,2017-01-11 00:03:07,"['news', 'Blue shark washes up on St Kilda bea...","127,157,120,160,104,162,161,171,206",Desktop,news,Blue shark washes up on St Kilda beach


In [4]:
newlist [['DimDate','StartTime']]=newlist['DateTime'].str.split(' ',expand=True)

In [5]:
newlist [['YEAR','MONTH','DAY']]=newlist['DimDate'].str.split('-',expand=True)

In [6]:
newlist [['HOUR','MINUTE','SECOND']]=newlist['StartTime'].str.split(':',expand=True)

In [7]:
newlist.rename(columns={'Platform':'DimPlatform'}, inplace = True)


In [2]:
newlist.to_csv('washed_video.csv',index=False)

NameError: name 'newlist' is not defined

In [155]:
newlist = pd.read_csv('washed_video.csv')

In [156]:
newlist.head(5)

,DateTime,VideoTitle,events,DimPlatform,DimSite,DimVideo,DimDate,StartTime,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND
0,2017-01-11 00:00:53,"['news', ' Shark attacks spearfisherman']","127,157,120,160,104,162,161,171,206",Desktop,news,Shark attacks spearfisherman,2017-01-11,00:00:53,2017,1,11,0,0,53
1,2017-01-11 00:03:51,"['news', ' Chilean navy films UFO']","127,157,120,160,104,162,161,171,206",Desktop,news,Chilean navy films UFO,2017-01-11,00:03:51,2017,1,11,0,3,51
2,2017-01-11 00:02:42,"['news', ' Chilean navy films UFO']","157,120,160,104,162,161,171,206",Desktop,news,Chilean navy films UFO,2017-01-11,00:02:42,2017,1,11,0,2,42
3,2017-01-11 00:04:07,"['news', 'Exclusive CCTV footage of Minto atta...","157,120,160,104,162,161,171,206",Desktop,news,Exclusive CCTV footage of Minto attack released,2017-01-11,00:04:07,2017,1,11,0,4,7
4,2017-01-11 00:03:07,"['news', 'Blue shark washes up on St Kilda bea...","127,157,120,160,104,162,161,171,206",Desktop,news,Blue shark washes up on St Kilda beach,2017-01-11,00:03:07,2017,1,11,0,3,7


In [163]:
df=pd.DataFrame(newlist.DimVideo.unique())

In [174]:
df.rename(columns={0:'Video_title'},inplace=True)

In [179]:
df['DimVideo_id'] = pd.DataFrame(np.arange(1,len(df)+1))

In [181]:
df.to_csv('DimVideo1.csv',index=False)

In [35]:
df = pd.DataFrame(newlist['DimSite'].unique())

In [36]:
import numpy as np

In [39]:
df = df.rename(columns={0:'DimSite'})

In [43]:
df['DimSite_id'] = pd.DataFrame(np.arange(1,6))

In [45]:
df.to_csv('DimSite_id.csv',index=False)

In [46]:
df

,DimSite,DimSite_id
0,news,1
1,App Web,2
2,App Android,3
3,App iPhone,4
4,App iPad,5


In [49]:
newlist = pd.read_csv('washed_video.csv')

In [52]:
newlist['DateTime'].nunique()

77776

In [53]:
df[['DateTime']]

(498109,)

In [54]:
newlist.head(2)

,DateTime,VideoTitle,events,DimPlatform,DimSite,DimVideo,DimDate,StartTime,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND
0,2017-01-11 00:00:53,"['news', ' Shark attacks spearfisherman']","127,157,120,160,104,162,161,171,206",Desktop,news,Shark attacks spearfisherman,2017-01-11,00:00:53,2017,1,11,0,0,53
1,2017-01-11 00:03:51,"['news', ' Chilean navy films UFO']","127,157,120,160,104,162,161,171,206",Desktop,news,Chilean navy films UFO,2017-01-11,00:03:51,2017,1,11,0,3,51


In [58]:
x

In [126]:
df = newlist[['DateTime','DimDate','StartTime','YEAR','quarter','MONTH','DAY','Weekday','HOUR','MINUTE']]

In [82]:
#星期一是0  星期三是2
newlist['Weekday'] = newlist['DimDate'].dt.weekday

In [124]:
#如果 newlist.Weekday的值等于 2 就换成’wednesday'
newlist.Weekday[newlist['Weekday']==2]='Wednesday'

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [149]:
df['DimDate'] = pd.to_datetime(df['DimDate'],format = "%Y/%m/%d")

In [151]:
df.to_csv('dimDate1.csv',index=False)

In [154]:

df = pd.read_csv('dimDate1.csv')
df

,DateTime,DimDate,StartTime,YEAR,quarter,MONTH,DAY,Weekday,HOUR,MINUTE
0,2017-01-11 00:00:53,2017-01-11,00:00:53,2017,1,1,11,Wednesday,0,0
1,2017-01-11 00:03:51,2017-01-11,00:03:51,2017,1,1,11,Wednesday,0,3
2,2017-01-11 00:02:42,2017-01-11,00:02:42,2017,1,1,11,Wednesday,0,2
3,2017-01-11 00:04:07,2017-01-11,00:04:07,2017,1,1,11,Wednesday,0,4
4,2017-01-11 00:03:07,2017-01-11,00:03:07,2017,1,1,11,Wednesday,0,3
...,...,...,...,...,...,...,...,...,...,...
498104,2017-01-11 23:46:00,2017-01-11,23:46:00,2017,1,1,11,Wednesday,23,46
498105,2017-01-11 23:59:46,2017-01-11,23:59:46,2017,1,1,11,Wednesday,23,59
498106,2017-01-11 23:57:57,2017-01-11,23:57:57,2017,1,1,11,Wednesday,23,57
498107,2017-01-11 23:56:13,2017-01-11,23:56:13,2017,1,1,11,Wednesday,23,56


In [70]:
newlist['DimDate'] =pd.to_datetime(newlist['DimDate'])

In [78]:
newlist['DimDate'].nunique()

1

In [83]:
newlist

,DateTime,VideoTitle,events,DimPlatform,DimSite,DimVideo,DimDate,StartTime,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,quarter,Weekday
0,2017-01-11 00:00:53,"['news', ' Shark attacks spearfisherman']","127,157,120,160,104,162,161,171,206",Desktop,news,Shark attacks spearfisherman,2017-01-11,00:00:53,2017,1,11,0,0,53,1,2
1,2017-01-11 00:03:51,"['news', ' Chilean navy films UFO']","127,157,120,160,104,162,161,171,206",Desktop,news,Chilean navy films UFO,2017-01-11,00:03:51,2017,1,11,0,3,51,1,2
2,2017-01-11 00:02:42,"['news', ' Chilean navy films UFO']","157,120,160,104,162,161,171,206",Desktop,news,Chilean navy films UFO,2017-01-11,00:02:42,2017,1,11,0,2,42,1,2
3,2017-01-11 00:04:07,"['news', 'Exclusive CCTV footage of Minto atta...","157,120,160,104,162,161,171,206",Desktop,news,Exclusive CCTV footage of Minto attack released,2017-01-11,00:04:07,2017,1,11,0,4,7,1,2
4,2017-01-11 00:03:07,"['news', 'Blue shark washes up on St Kilda bea...","127,157,120,160,104,162,161,171,206",Desktop,news,Blue shark washes up on St Kilda beach,2017-01-11,00:03:07,2017,1,11,0,3,7,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498104,2017-01-11 23:46:00,"['App iPad', 'Episode', 'heartbeat;season-11',...","501,508,507,509,121,105,120,160,162,171,206,15...",Ipad,App iPad,Ep 11 A Gentlemen's Sport,2017-01-11,23:46:00,2017,1,11,23,46,0,1,2
498105,2017-01-11 23:59:46,"['App iPad', 'Live', 'Channel 9', 'WA']","501,508,507,509,121,105,120,160,162,171,206,15...",Ipad,App iPad,WA,2017-01-11,23:59:46,2017,1,11,23,59,46,1,2
498106,2017-01-11 23:57:57,"['App iPad', 'Episode', 'hawaii-life;season-7'...","501,508,507,509,121,105,120,160,162,171,206,15...",Ipad,App iPad,Ep 4 Life-Changing Move to Oahu,2017-01-11,23:57:57,2017,1,11,23,57,57,1,2
498107,2017-01-11 23:56:13,"['App iPhone', 'Clip', 'a-current-affair;2017'...","501,508,507,509,121,105,120,160,162,171,206,15...",iPhone,App iPhone,Cruise into retirement,2017-01-11,23:56:13,2017,1,11,23,56,13,1,2


In [87]:
import pandas as pd
newlist = pd.read_csv('washed_video.csv')

# ====VideoPlay table ====

In [90]:
newlist.shape

(498109, 14)

In [91]:
VideoPlay =newlist.drop(columns=['VideoTitle','VideoTitle','SECOND'])
VideoPlay['events']=VideoPlay['events'].apply(lambda x:x.replace(',','/'))

In [92]:
VideoPlay.head(2)

,DateTime,events,DimPlatform,DimSite,DimVideo,DimDate,StartTime,YEAR,MONTH,DAY,HOUR,MINUTE
0,2017-01-11 00:00:53,127/157/120/160/104/162/161/171/206,Desktop,news,Shark attacks spearfisherman,2017-01-11,00:00:53,2017,1,11,0,0
1,2017-01-11 00:03:51,127/157/120/160/104/162/161/171/206,Desktop,news,Chilean navy films UFO,2017-01-11,00:03:51,2017,1,11,0,3


# ====Dimdate table ======

In [27]:
dimDate = pd.read_csv('dimDate1.csv')
dimDate.head(2)

,DateTime,DimDate,StartTime,YEAR,quarter,MONTH,DAY,Weekday,HOUR,MINUTE
0,2017-01-11 00:00:53,2017-01-11,00:00:53,2017,1,1,11,Wednesday,0,0
1,2017-01-11 00:03:51,2017-01-11,00:03:51,2017,1,1,11,Wednesday,0,3


In [4]:
dimDate['DateTime'].nunique()

77776

In [30]:
dimDate = pd.DataFrame(dimDate['DateTime'].unique(),columns=['FullDate'])
dimDate

,FullDate
0,2017-01-11 00:00:53
1,2017-01-11 00:03:51
2,2017-01-11 00:02:42
3,2017-01-11 00:04:07
4,2017-01-11 00:03:07
...,...
77771,2017-01-11 23:49:02
77772,2017-01-11 23:37:46
77773,2017-01-11 23:47:43
77774,2017-01-11 23:48:17


In [31]:
dimDate[['Date','Times']] = dimDate['FullDate'].str.split(' ',expand=True)

In [32]:
dimDate[['Year','Month','Day']] = dimDate['Date'].str.split('-',expand=True)
dimDate[['Hour','Minute','Second']] = dimDate['Times'].str.split(':',expand=True)

In [37]:
dimDate['FullDate'] = pd.to_datetime(dimDate['FullDate'],format = "%Y-%m-%d %H:%M:%S")

In [38]:
# 这个column是 datetime类型 才可以使用 .dt.querater
dimDate['Quarter'] = dimDate['FullDate'].dt.quarter

In [51]:
dimDate['Weekday'] = dimDate['FullDate'].dt.weekday

In [42]:
dimDate['Weekday'][dimDate['Weekday'] == 2] = 'Wednesday'

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
dimDate

,FullDate,Date,Times,Year,Month,Day,Hour,Minute,Second,Quarter,Weekday
0,2017-01-11 00:00:53,2017-01-11,00:00:53,2017,01,11,00,00,53,1,2
1,2017-01-11 00:03:51,2017-01-11,00:03:51,2017,01,11,00,03,51,1,2
2,2017-01-11 00:02:42,2017-01-11,00:02:42,2017,01,11,00,02,42,1,2
3,2017-01-11 00:04:07,2017-01-11,00:04:07,2017,01,11,00,04,07,1,2
4,2017-01-11 00:03:07,2017-01-11,00:03:07,2017,01,11,00,03,07,1,2
...,...,...,...,...,...,...,...,...,...,...,...
77771,2017-01-11 23:49:02,2017-01-11,23:49:02,2017,01,11,23,49,02,1,2
77772,2017-01-11 23:37:46,2017-01-11,23:37:46,2017,01,11,23,37,46,1,2
77773,2017-01-11 23:47:43,2017-01-11,23:47:43,2017,01,11,23,47,43,1,2
77774,2017-01-11 23:48:17,2017-01-11,23:48:17,2017,01,11,23,48,17,1,2


In [71]:
dimDate['Weekday'].replace(2,'Wednesday',inplace=True)

In [79]:
dim = dimDate[['FullDate','Date','Times','Year','Quarter','Month','Day','Weekday','Hour','Minute']]

In [104]:
dim.rename(columns={'FullDate':'DateTime'},inplace=True)
                                        

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [105]:
dim.to_csv('dimDate2.csv',index=False)

In [107]:
dimDate = pd.read_csv('dimDate2.csv')
dimDate.head(2)
dimDate.shape

(77776, 10)

# ====Dimplatform table ====

In [94]:
dimPlatform = pd.read_csv('DimPlatform_id.csv')
dimPlatform.head(2)

,DimPlatform,DimPlatform_id
0,Desktop,1
1,iPhone,2


# =====DimSite table =====

In [95]:
dimSite = pd.read_csv('DimSite_id.csv')
dimSite.head(2)

,DimSite,DimSite_id
0,news,1
1,App Web,2


# =====DimVideo table ====

In [96]:
dimVideo = pd.read_csv('DimVideo1.csv')
dimVideo.head(2)

,Video_title,DimVideo_id
0,Shark attacks spearfisherman,1
1,Chilean navy films UFO,2


In [116]:
dimVideo.rename(columns={'Video_title':'DimVideo'},inplace=True)

In [117]:
dimVideo.head(2)

,DimVideo,DimVideo_id
0,Shark attacks spearfisherman,1
1,Chilean navy films UFO,2


# ========== join ========
# ========== join ========
# ========== join ========

In [ ]:
#====videoPlay + dimdate====

In [108]:
videoPlay_dimdate = pd.merge(VideoPlay,dimDate,on='DateTime',how='outer')
videoPlay_dimdate.head(2)

,DateTime,events,DimPlatform,DimSite,DimVideo,DimDate,StartTime,YEAR,MONTH,DAY,...,MINUTE,Date,Times,Year,Quarter,Month,Day,Weekday,Hour,Minute
0,2017-01-11 00:00:53,127/157/120/160/104/162/161/171/206,Desktop,news,Shark attacks spearfisherman,2017-01-11,00:00:53,2017,1,11,...,0,2017-01-11,00:00:53,2017,1,1,11,Wednesday,0,0
1,2017-01-11 00:00:53,157/120/160/104/162/161/171/206,Desktop,news,Australian couple ‘stranded’ on glacier in Ic...,2017-01-11,00:00:53,2017,1,11,...,0,2017-01-11,00:00:53,2017,1,1,11,Wednesday,0,0


# =====videoPlay_dimdate + Dimplatform ===

In [109]:
videoPlay_dimdate_dimplatform = pd.merge(videoPlay_dimdate,dimPlatform,on='DimPlatform',how='outer')
videoPlay_dimdate_dimplatform.head()

,DateTime,events,DimPlatform,DimSite,DimVideo,DimDate,StartTime,YEAR,MONTH,DAY,...,Date,Times,Year,Quarter,Month,Day,Weekday,Hour,Minute,DimPlatform_id
0,2017-01-11 00:00:53,127/157/120/160/104/162/161/171/206,Desktop,news,Shark attacks spearfisherman,2017-01-11,00:00:53,2017,1,11,...,2017-01-11,00:00:53,2017,1,1,11,Wednesday,0,0,1
1,2017-01-11 00:00:53,157/120/160/104/162/161/171/206,Desktop,news,Australian couple ‘stranded’ on glacier in Ic...,2017-01-11,00:00:53,2017,1,11,...,2017-01-11,00:00:53,2017,1,1,11,Wednesday,0,0,1
2,2017-01-11 00:00:53,127/157/120/160/104/162/161/171/206,Desktop,news,Avalanche buries hotel guests in back of TV i...,2017-01-11,00:00:53,2017,1,11,...,2017-01-11,00:00:53,2017,1,1,11,Wednesday,0,0,1
3,2017-01-11 00:00:53,157/120/160/104/162/161/171/206,Desktop,news,Sailors jump to safety after waves smash yacht...,2017-01-11,00:00:53,2017,1,11,...,2017-01-11,00:00:53,2017,1,1,11,Wednesday,0,0,1
4,2017-01-11 00:03:51,127/157/120/160/104/162/161/171/206,Desktop,news,Chilean navy films UFO,2017-01-11,00:03:51,2017,1,11,...,2017-01-11,00:03:51,2017,1,1,11,Wednesday,0,3,1


# =====join  videoPlay_dimdate + Dimplatform +  DimSite ===

In [110]:
videoPlay_dimdate_Dimplatform_DimSite = pd.merge(videoPlay_dimdate_dimplatform,dimSite,on = 'DimSite',how='outer')
videoPlay_dimdate_Dimplatform_DimSite.head(2)

,DateTime,events,DimPlatform,DimSite,DimVideo,DimDate,StartTime,YEAR,MONTH,DAY,...,Times,Year,Quarter,Month,Day,Weekday,Hour,Minute,DimPlatform_id,DimSite_id
0,2017-01-11 00:00:53,127/157/120/160/104/162/161/171/206,Desktop,news,Shark attacks spearfisherman,2017-01-11,00:00:53,2017,1,11,...,00:00:53,2017,1,1,11,Wednesday,0,0,1,1
1,2017-01-11 00:00:53,157/120/160/104/162/161/171/206,Desktop,news,Australian couple ‘stranded’ on glacier in Ic...,2017-01-11,00:00:53,2017,1,11,...,00:00:53,2017,1,1,11,Wednesday,0,0,1,1


# =====join videoPlay_dimdate + Dimplatform +  DimSite + DimVideo===

In [118]:
videoPlay_dimdate_Dimplatform_DimSite_DimVideo = pd.merge(videoPlay_dimdate_Dimplatform_DimSite,dimVideo,on ='DimVideo',how ='outer')
videoPlay_dimdate_Dimplatform_DimSite_DimVideo.head(2)


,DateTime,events,DimPlatform,DimSite,DimVideo,DimDate,StartTime,YEAR,MONTH,DAY,...,Year,Quarter,Month,Day,Weekday,Hour,Minute,DimPlatform_id,DimSite_id,DimVideo_id
0,2017-01-11 00:00:53,127/157/120/160/104/162/161/171/206,Desktop,news,Shark attacks spearfisherman,2017-01-11,00:00:53,2017,1,11,...,2017,1,1,11,Wednesday,0,0,1,1,1
1,2017-01-11 00:13:09,157/120/160/104/162/161/171/206,Desktop,news,Shark attacks spearfisherman,2017-01-11,00:13:09,2017,1,11,...,2017,1,1,11,Wednesday,0,13,1,1,1


In [122]:
fact_table =videoPlay_dimdate_Dimplatform_DimSite_DimVideo.loc[:,['DateTime','DimPlatform_id','DimSite_id','DimVideo_id','events']]
fact_table.shape
                                                            

(498109, 5)

In [121]:
fact_table.to_csv('factTable_VideoPlay.csv',index=False)

# ======try  add events ====

In [48]:
try_events =newlist.drop(columns=['VideoTitle','VideoTitle','SECOND'])

In [51]:
try_events['events']=try_events['events'].apply(lambda x:x.replace(',',' '))

In [53]:
try_events.head(2)


,DateTime,events,DimPlatform,DimSite,DimVideo,DimDate,StartTime,YEAR,MONTH,DAY,HOUR,MINUTE
0,2017-01-11 00:00:53,127 157 120 160 104 162 161 171 206,Desktop,news,Shark attacks spearfisherman,2017-01-11,00:00:53,2017,1,11,0,0
1,2017-01-11 00:03:51,127 157 120 160 104 162 161 171 206,Desktop,news,Chilean navy films UFO,2017-01-11,00:03:51,2017,1,11,0,3


# ======展示

In [ ]:
# outer join tables
videoPlay_dimdate = pd.merge(VideoPlay,dimDate,on='DateTime',how='outer')
videoPlay_dimdate_dimplatform = pd.merge(videoPlay_dimdate,dimPlatform,on='DimPlatform',how='outer')
videoPlay_dimdate_Dimplatform_DimSite = pd.merge(videoPlay_dimdate_dimplatform,dimSite,on = 'DimSite',how='outer')
videoPlay_dimdate_Dimplatform_DimSite_DimVideo = pd.merge(videoPlay_dimdate_Dimplatform_DimSite,dimVideo,on ='DimVideo',how ='outer')


In [70]:
# extract the columns for the fact table
fact_table =videoPlay_dimdate_Dimplatform_DimSite_DimVideo.loc[:,['DateTime','DimPlatform_id','DimSite_id','DimVideo_id','events']]
fact_table.head(100)

,DateTime,DimPlatform_id,DimSite_id,DimVideo_id,events
0,2017-01-11 00:00:53,1,1,1,127/157/120/160/104/162/161/171/206
1,2017-01-11 00:00:53,1,1,1,127/157/120/160/104/162/161/171/206
2,2017-01-11 00:00:53,1,1,1,127/157/120/160/104/162/161/171/206
3,2017-01-11 00:00:53,1,1,1,127/157/120/160/104/162/161/171/206
4,2017-01-11 00:00:53,1,1,1,127/157/120/160/104/162/161/171/206
...,...,...,...,...,...
95,2017-01-11 00:03:31,1,1,1,157/120/160/104/162/161/171/206
96,2017-01-11 00:03:31,1,1,1,157/120/160/104/162/161/171/206
97,2017-01-11 00:03:31,1,1,1,157/120/160/104/162/161/171/206
98,2017-01-11 00:03:31,1,1,1,157/120/160/104/162/161/171/206


In [77]:
fact_table['DimPlatform_id'].unique()

array([1, 2, 4, 3])

In [75]:
fact_table['DimSite_id'].unique()

array([1, 2, 4, 5, 3])